In [ ]:
import pandas as pd
import os

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import transforms as T
from tqdm.notebook import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
df_train = pd.read_csv('competition_train.csv')
df_train = df_train.loc[~df_train['BoxesString'].str.contains('no_box'),:].reset_index(drop=True)
df_val = pd.read_csv('competition_val.csv')
df_val = df_val.loc[~df_val['BoxesString'].str.contains('no_box'),:].reset_index(drop=True)
df_test = pd.read_csv('competition_test.csv')
df_test = df_test.loc[~df_test['BoxesString'].str.contains('no_box'),:].reset_index(drop=True)
print(df_train)

In [ ]:
def convert_boxes_string_to_list(boxes_string):
    groups = boxes_string.split(";")
    result = []
    for group in groups:
        nums = group.split()
        # Filtrar valores que no son números
        nums = [int(num) for num in nums if num.isdigit()]
        result.append(nums)
    return result

# Aplica la función a la columna 'BoxesString'
df_test['BoxesString'] = df_test['BoxesString'].apply(convert_boxes_string_to_list)
df_train['BoxesString'] = df_train['BoxesString'].apply(convert_boxes_string_to_list)
df_val['BoxesString'] = df_val['BoxesString'].apply(convert_boxes_string_to_list)

print(df_train.head())



In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, root_dir):
        self.df = df
        self.root_dir = root_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = T.ToTensor()(Image.open(img_name).convert("RGB"))/255
        boxes = self.df.iloc[idx, 1]
        num_objs = len(boxes)
        boxes = torch.tensor(boxes, dtype=torch.float32)

        labels = torch.ones((len(boxes),), dtype=torch.int64)

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
            "area": area,
            "iscrowd": iscrowd
        }

        return image, target

    def print_image(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name)
        draw = ImageDraw.Draw(image)
        boxes = self.df.iloc[idx, 1]
        for box in boxes:
            draw.rectangle(box, outline="red", width=4)
        plt.imshow(image)
        plt.show()

In [ ]:
dataset_train = ImageDataset(df_train, 'images')
dataset_val = ImageDataset(df_val, 'images')
dataset_test = ImageDataset(df_test, 'images')

dataset_val.print_image(0)

In [ ]:
model = fasterrcnn_resnet50_fpn(progress=True, pretrained_backbone=True, num_classes=2, box_detections_per_img=200)
model.to(device)



optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets

data_loader_train = DataLoader(dataset_train, batch_size=8, collate_fn=collate_fn, shuffle=True)
len(data_loader_train)

In [ ]:
class EarlyStopping:
    def __init__(self, patience):
        self.best = 10
        self.count = 0
        self.patience = patience

    def earlyStopping(self, num):
        if num <= self.best:
            self.best = num
            self.count = 0
        else:
            self.count += 1
        if self.count >= self.patience:
            return self.best
        return None

In [ ]:
model.train()
early_stopping = EarlyStopping(3)

losses_dict = {'loss_classifier': [], 'loss_box_reg': [], 'loss_objectness': [], 'loss_rpn_box_reg': [], 'total_loss': []}
stop_training = False

for epoch in tqdm(range(10), desc='epochs', colour='green'):
    if stop_training: 
        break
    
    losses_avg = {'loss_classifier': [], 'loss_box_reg': [], 'loss_objectness': [], 'loss_rpn_box_reg': [], 'total_loss': []}

    for images, targets in tqdm(data_loader_train, desc='dataloader', colour='blue'):
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

        loss_dict = model(images, targets)
        losses_avg['loss_classifier'].append(loss_dict['loss_classifier'].item())
        losses_avg['loss_box_reg'].append(loss_dict['loss_box_reg'].item())
        losses_avg['loss_objectness'].append(loss_dict['loss_objectness'].item())
        losses_avg['loss_rpn_box_reg'].append(loss_dict['loss_rpn_box_reg'].item())
        losses = sum(loss for loss in loss_dict.values())
        losses_avg['total_loss'].append(losses.item())



        # for loss in losses_dict['total_loss']:
        #     best = early_stopping.earlyStopping(loss)
        #     if best is not None:
        #         stop_training = True 
        #         break
        # if best is None:  
        #     best = early_stopping.best
        losses.backward()
        optimizer.step()
        optimizer.zero_grad()  
    losses_dict['loss_classifier'].append(sum(losses_avg['loss_classifier'])/len(losses_avg['loss_classifier']))
    losses_dict['loss_box_reg'].append(sum(losses_avg['loss_box_reg'])/len(losses_avg['loss_box_reg']))
    losses_dict['loss_objectness'].append(sum(losses_avg['loss_objectness'])/len(losses_avg['loss_objectness']))
    losses_dict['loss_rpn_box_reg'].append(sum(losses_avg['loss_rpn_box_reg'])/len(losses_avg['loss_rpn_box_reg']))
    losses_dict['total_loss'].append(sum(losses_avg['total_loss'])/len(losses_avg['total_loss']))
    print(f"Epoch {epoch + 1}, loss_classifier: {losses_dict['loss_classifier']}\n loss_box_reg: {losses_dict['loss_box_reg']}\n loss_objectness: {losses_dict['loss_objectness']}\n loss_rpn_box_reg: {losses_dict['loss_rpn_box_reg']}\n total_loss: {losses_dict['total_loss']}")